In [1]:
# Author: Zhengxiang (Jack) Wang 
# Date: 2022-01-20
# GitHub: https://github.com/jaaack-wang 

## Quick start

We used a lengthy tutorial to illustrate how to convert text dataset for training text classification models. However, if there are handy wrapped up functions, as available in many deep learning frameworks, this process can be easily done with a few lines of code. The following is an illustration, although in real projects, you many need additional lines of code to suit your specific needs. 

The following is a quick start. A more elaborated explanation is given afterwards.

In [2]:
from utils import *
import jieba

# ---- load dataset ----
train_set = load_dataset('train.tsv')

# ---- numericalize the train set ----
V = TextVectorizer(jieba.lcut) 
text = gather_text(train_set) # for collecting texts from train set
V.build_vocab(text) # for building mapping vocab_to_idx dictionary and text_encoder
train_set_encoded = list(encode_dataset(train_set, encoder=V.text_encoder)) # encodoing train set

# ---- build mini batches for the train set ----
train_set_batched = build_batches(train_set_encoded, batch_size=64, include_seq_len=True)
print(f"Number of {len(train_set_batched)} batches created!")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w9/d_nplhzj4qx35xxlgljgdtjh0000gn/T/jieba.cache
Loading model cost 0.709 seconds.
Prefix dict has been built successfully.


Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.
Number of 63 batches created!


## Intro to the wrapped functions

As we will not rewrite or copy and paste (a) same functions over and over again to just load and preprocess data throughout the tutorials, some wrapped functions are therefore provided. These wrapped functions can be seen in the `utils.py` file in the same folder.

These wrapped functions mostly come from the last tutorial (see: `2 - preprocess_data.ipynb`) with some revisions to make some functons more reusable. Let's do what we do in the last tutorial and this tutorial will introduce these wrapped up functions along the way.

## Load dataset

The wrapped up `load_dataset` allows:

- loading a dataset (for tutorials in this repository) given its filepath; 
- (extended) loading multiples datasets given their filepathes.

In [3]:
from utils import load_dataset

train_set, dev_set, test_set = load_dataset(['train.tsv', 'dev.tsv', 'test.tsv'])

# check. should be 4000, 1200, 1200 (recall `1 - get_data.ipynb`)
len(train_set), len(dev_set), len(test_set)

(4000, 1200, 1200)

## Numericalize text 

Recall that we need to encode the text data into something numerical so that we can train models on them. As elaborated in the last tutorial, to do so, we need to have a tokenizer and a related dictionary where we can map a token to an unique index. If we want to deocde the encoded text, we will also need to have a reversed dictionary that maps an index to a token. 

Re-doing all these can be tedious, so this tutorials introduces a highly wrapped up class function `TextVectorizer` to make everything easy. The `TextVectorizer` class can do the following: 

- building the `vocab_to_idx` and `idx_to_vocab` mapping dictionaries quickly given text and tokenizer. 
- encoding and decoding any given text(s) using the tokenizer used to build the dictionaries; 
- save and load the built `vocab_to_idx` and `idx_to_vocab` dictionaries for reuse; 

If you have a list of tokens ready that can be gotten from the tokenizer you pass to the `TextVectorizer` class, then you can also quickly build the dictionaries from that list of tokens. 

`TextVectorizer` is also callable, which means that after initializing it, you can directly encode text(s) by calling one(s) to it. 

### Initialization 

To initialize the `TextVectorizer`, you must pass a tokenizer function/method to it, which takes str as input and returns a list of tokens. You can also pass a text preprocessor function/method to it to preprocess a given text before it is being tokenized by the tokenizer. Of course, you can build a tokenizer that incorprates the preprocessor and only pass the tokenizer. They are a same thing. 

We will use the same tokenizer we used in the last tutorial, which is saved in the `utils.py` file.

In [4]:
from utils import TextVectorizer
import jieba


V = TextVectorizer(jieba.lcut)

### Building `vocab_to_idx` and `idx_to_vocab` dictionaries

To build these two dictionaries, all you need is just a (list) of text(s) and then pass it/them to the `TextVectorizer.build_vocab` function. If you have a list of tokens, you can also do so by passing the tokens to `TextVectorizer.build_vocab_from_list_tks`.

If you choose to use `TextVectorizer.build_vocab`, you can also choose whether to build the vocab randomly or based on the occurences of the tokens in descending order. By defaults, it will build the vocab basde on the latter. When this is the case, you can specify how many most frequnt tokens you want to keep by specifying, for example, `top=10000`. This is not an option for the random mode because that does not make sense. 

To gather the text, we will use a simple function `gather_text` that specifically gather all the "text_a" and "text_b" from our datasets into a list. 

**Remember that the dictionaries should be built upon the train set or some external source, but never on the dev set or test set.**

In [5]:
from utils import gather_text

text = gather_text(train_set)
print(f"{len(text)} pieces of texts gathered from the train set.\n")

V.build_vocab(text)

4000 pieces of texts gathered from the train set.

Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.


In [6]:
from random import sample, seed
# check

seed(543)

tmp = "{:20}{}"
print("5 random examples from the vocab_to_idx dictionary\n")

for item in sample(list(V.vocab_to_idx.items()), 5):
    print(tmp.format(*item))
    
    
print("\n\n5 random examples from the idx_to_vocab dictionary\n")

for idx, tk in sample(list(V.idx_to_vocab.items()), 5):
    print(tmp.format(str(idx), tk))

5 random examples from the vocab_to_idx dictionary

毛巾被                 21035
微微                  4131
冲突                  2966
卡瓦                  15267
端口                  19463


5 random examples from the idx_to_vocab dictionary

6066                差错
9083                听不见
2760                强悍
18630               鸭绒被
2321                套


### Saving the mapping dictionaries

You can use `TextVectorizer.save_vocab_as_json` which by default will save the dictionaries, if built, as `vocab_to_idx.json` and `idx_to_vocab.json` in the current working directory. 

In [7]:
V.save_vocab_as_json()

vocab_to_idx.json has been successfully saved!
idx_to_vocab.json has been successfully saved!


### Reusing the mapping dictionaries

When you need to reuse them, simply call `TextVectorizer.load_vocab_from_json`. If you do specify the filepathes to the two dictionaries, it will search for `vocab_to_idx.json` and `idx_to_vocab.json` in the current working directory and return them if any of them exists. 

Below, we first empty the two mapping dictionaries and reload them from the json files we saved just now. 

In [8]:
V.vocab_to_idx, V.idx_to_vocab = None, None

V.load_vocab_from_json()

vocab_to_idx.json has been successfully loaded! Please call X.vocab_to_idx to find out more.
idx_to_vocab.json has been successfully loaded! Please call X.idx_to_vocab to find out more.

Where [X] stands for the name you used for this TextVectorizer class.


### Encoding and decoding text

- Encoding: call initialized `TextVectorizer` or `TextVectorizer.text_encoder`.
- Decoding: call `TextVectorizer.text_decoder`.

Let's use some sample texts from the dev set as examples.

In [9]:
seed(894)

dev_text = gather_text(dev_set)
sample_texts = sample(dev_text, 2)

assert len(dev_text) == len(dev_set)

for text in sample_texts:
    encoded = V.text_encoder(text) # or V(text)
    decoded = V.text_decoder(encoded, sep='')
    print("Original:", text)
    print("Encoded:", encoded)
    print("Decoded:", decoded)
    print()
    
# Or pass a list of text or text_ids to encode or decode
print(V.text_decoder(V(sample_texts), sep=''))

Original: 酒店的地理位置还算便利，但是酒店的设施非常成旧。前台一听说是携程订的，给的房间全部是陈旧的客房，设施很差，但房价很高。和当地朋友一说，他们认为价格比他们去订高多了！唉……
Encoded: [13, 3, 478, 21, 259, 964, 2, 58, 13, 3, 133, 53, 17157, 4, 95, 147, 933, 7, 122, 373, 3, 2, 40, 3, 22, 1010, 7, 392, 3, 399, 2, 133, 467, 2, 44, 495, 10, 157, 4, 25, 1060, 131, 4880, 2, 139, 559, 1576, 139, 1, 1, 5, 11, 1246, 183, 183]
Decoded: 酒店的地理位置还算便利，但是酒店的设施非常成旧。前台一听说是携程订的，给的房间全部是陈旧的客房，设施很差，但房价很高。和当地朋友一说，他们认为价格比他们[UNK][UNK]了！唉……

Original: 有点厚重。显卡不是独立的，是集显，如果玩大型网游的，建议买独显的。
Encoded: [75, 3544, 4, 449, 66, 1485, 3, 2, 7, 17327, 2, 111, 868, 3035, 1, 3, 2, 185, 38, 1570, 3, 4]
Decoded: 有点厚重。显卡不是独立的，是集显，如果玩大型[UNK]的，建议买独显的。

['酒店的地理位置还算便利，但是酒店的设施非常成旧。前台一听说是携程订的，给的房间全部是陈旧的客房，设施很差，但房价很高。和当地朋友一说，他们认为价格比他们[UNK][UNK]了！唉……', '有点厚重。显卡不是独立的，是集显，如果玩大型[UNK]的，建议买独显的。']


### Encode the train set

We will reuse the `encode_dataset` built in the last tutorial to encode the train set (or dev set for validation).

In [10]:
from utils import encode_dataset

train_set_encoded = list(encode_dataset(train_set, encoder=V.text_encoder))

# check
print(train_set_encoded[0])

[[188, 6965, 1177, 3, 428, 37, 99, 2, 17, 10681, 10682, 343, 913, 1875, 2, 604, 2299, 27, 10683, 27, 2133, 27, 1178, 27, 10684, 5392, 4, 13, 290, 76, 2, 44, 21, 259, 1048, 4, 6, 2685, 14, 284, 3, 4400, 2, 1089, 422, 2, 96, 397, 497, 7, 59, 4, 6, 531, 3, 108, 7, 2686, 3, 2, 21, 259, 471, 4, 6, 43, 319, 2, 76], 1]


## Building batches

We will reuse the `build_batches` built in the last tutorial to build the mini batches for the train set (or dev set for validation). The parameters are same except one `include_seq_len` that is added specifically for 

In [11]:
from utils import build_batches

train_set_batched = build_batches(train_set_encoded, batch_size=64, include_seq_len=True)

print(f"Number of {len(train_set_batched)} batches created!")

Number of 63 batches created!


In [12]:
# check

for idx, batch in enumerate(train_set_batched[:5]):
    text, seq_len, label = batch
    print(f"{'-' * 10} # {idx+1} batch {'-' * 10}")
    
    print("Shape of text batch:", text.shape)
    print("Shape of seq_len batch:", seq_len.shape)
    print("Shape of label batch:", label.shape)
    print()

---------- # 1 batch ----------
Shape of text batch: (64, 325)
Shape of seq_len batch: (64,)
Shape of label batch: (64,)

---------- # 2 batch ----------
Shape of text batch: (64, 291)
Shape of seq_len batch: (64,)
Shape of label batch: (64,)

---------- # 3 batch ----------
Shape of text batch: (64, 163)
Shape of seq_len batch: (64,)
Shape of label batch: (64,)

---------- # 4 batch ----------
Shape of text batch: (64, 247)
Shape of seq_len batch: (64,)
Shape of label batch: (64,)

---------- # 5 batch ----------
Shape of text batch: (64, 334)
Shape of seq_len batch: (64,)
Shape of label batch: (64,)

